In [1]:
import pandas as pd
import numpy as np
from joblib import dump, load
import random

from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.inspection import permutation_importance
from scipy.stats import ttest_rel, friedmanchisquare
from statsmodels.stats.multicomp import pairwise_tukeyhsd

Helper Functions

In [2]:
# initial random seed
seed = 42

In [3]:
def compute_metrics(y_true, y_pred):
    """Compute RMSE, MAE, and R2."""
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return rmse, mae, r2

In [4]:
def compute_stability(model, X, y, noise_level=0.01, n_trials=5):
    """
    Add Gaussian noise to numeric features and measure average relative RMSE change.
    noise_level is fraction of std-dev of each feature.
    """
    numeric_cols = X.select_dtypes(include=[np.number]).columns
    base_rmse = np.sqrt(mean_squared_error(y, model.predict(X)))
    rel_changes = []
    
    for _ in range(n_trials):
        Xp = X.copy()
        noise = np.random.normal(0, noise_level * Xp[numeric_cols].std(), 
                                 size=Xp[numeric_cols].shape)
        Xp[numeric_cols] += noise
        rp = model.predict(Xp)
        rmse_p = np.sqrt(mean_squared_error(y, rp))
        rel_changes.append((rmse_p - base_rmse) / base_rmse)
    
    # Return average relative change (lower = more stable)
    return np.mean(rel_changes)

# Model Definition

In [5]:
# models = {
#     'Ridge': Ridge(random_state=42, alpha=1.0), 
#     'RandomForest': RandomForestRegressor(n_estimators=100, random_state=seed),
#     'XGBoost': XGBRegressor(
#         n_estimators=100, 
#         eval_metric='rmse', 
#         random_state=seed,
#         learning_rate=0.1,
#         ),
#     'MLP': MLPRegressor(
#         hidden_layer_sizes=(128, 64, 32),
#         activation='relu',
#         solver='lbfgs',
#         learning_rate='adaptive',
#         learning_rate_init=1e-3,
#         alpha=1e-4,
#         early_stopping=True,
#         validation_fraction=0.1,
#         max_iter=300,
#         random_state=seed
#         )
# }

def init_models(seed):
    models = {
        'Ridge': Ridge(random_state=42, alpha=1.0), 
        'RandomForest': RandomForestRegressor(n_estimators=100, random_state=seed),
        'XGBoost': XGBRegressor(
            n_estimators=100, 
            eval_metric='rmse', 
            random_state=seed,
            learning_rate=0.1,
            ),
        'MLP': MLPRegressor(
            hidden_layer_sizes=(128, 64, 32),
            activation='relu',
            solver='lbfgs',
            learning_rate='adaptive',
            learning_rate_init=1e-3,
            alpha=1e-4,
            early_stopping=True,
            validation_fraction=0.1,
            max_iter=300,
            random_state=seed
            )
    }

    return models

In [6]:
param_grids = {
    'Ridge': {
        'alpha': [0.01, 0.1, 1.0, 10.0, 100.0]
        },
    'RandomForest': {
        'n_estimators': [100, 200],
        'max_depth': [None, 10, 20]
        },
    'XGBoost': {
        'n_estimators': [100, 200],
        'learning_rate': [0.01, 0.1],
        'max_depth': [3, 6],
        'subsample': [0.8, 1.0],
        },
    'MLP': {
        # deeper, tapered architectures
        'hidden_layer_sizes': [
            (128, 64, 32),
            (256, 128, 64, 32)
        ],
        'alpha': [1e-5, 1e-4, 1e-3],
        'learning_rate_init': [1e-4, 1e-3],
        'activation': ['relu', 'tanh'],
        'solver': ['adam', 'lbfgs'],
        'early_stopping': [True],
        'max_iter': [500]
    }
}

In [7]:
datasets = load('..\data\experimental\experiment_datasets_2.joblib')  # Load datasets from joblib file

# Manual Training

In [6]:
train_df = datasets['baselines']['full']['within_sample']['within_sample']['train']
test_df = datasets['baselines']['full']['within_sample']['within_sample']['test']

train_df.shape, test_df.shape

((291, 82), (73, 82))

In [7]:
X_train = train_df.drop(columns=['totalEsg'])
y_train = train_df['totalEsg']
X_test  = test_df.drop(columns=['totalEsg'])
y_test  = test_df['totalEsg']

In [8]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((291, 81), (291,), (73, 81), (73,))

In [9]:
model = models['Ridge']  # Choose the model you want to use

In [10]:
model

Ridge(random_state=42)

In [11]:
model.fit(X_train, y_train)

l:\Programs\anaconda\envs\esg_pred\Lib\site-packages\sklearn\linear_model\_ridge.py:215: LinAlgWarning: Ill-conditioned matrix (rcond=5.17923e-28): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


Ridge(random_state=42)

In [12]:
# predict
y_pred = model.predict(X_test)

In [13]:
# compute metrics
rmse, mae, r2 = compute_metrics(y_test, y_pred)
stability = compute_stability(model, X_test, y_test,
                            noise_level=0.01, n_trials=5)

In [14]:
manual_results = {
            'RMSE': rmse,
            'MAE': mae,
            'R2': r2,
            'Stability': stability
        }

In [15]:
pd.DataFrame(manual_results, index=[0])

,RMSE,MAE,R2,Stability
0,44.795695,15.571981,-32.245542,-0.001084


# Iterative Training

In [8]:
def train_and_evaluate(model_name, model, X_train, y_train, X_test, y_test, param_grid):
    """Train model with grid search and evaluate performance"""
    # Prepare grid search
    grid = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        cv=3,
        scoring='neg_root_mean_squared_error',
        n_jobs=-1
    )
    
    # Train model
    grid.fit(X_train, y_train)
    best_model = grid.best_estimator_
    
    # Make predictions
    y_pred = best_model.predict(X_test)
    
    # Compute metrics
    rmse, mae, r2 = compute_metrics(y_test, y_pred)
    stability = compute_stability(best_model, X_test, y_test, 
                                noise_level=0.01, n_trials=5)
    
    # Feature importance (if applicable)
    if hasattr(best_model, "feature_importances_"):
        importances = best_model.feature_importances_
    elif hasattr(best_model, "coef_"):
        importances = best_model.coef_.ravel()
    else:
        perm = permutation_importance(
            best_model, X_test, y_test,
            n_repeats=5, random_state=0, n_jobs=-1
        )
        importances = perm.importances_mean
    
    return {
        'model': model_name,
        'RMSE': rmse,
        'MAE': mae,
        'R2': r2,
        'Stability': stability,
        'Feature_names': list(X_train.columns),
        'Feature_importances': importances.tolist(),
    }

In [9]:
def prepare_data(df, target_col='totalEsg'):
    """Separate features and target"""
    X = df.drop(columns=[target_col])
    y = df[target_col]
    return X, y

In [10]:
def process_dataset(train_df, test_df, models, param_grids, scenario_info):
    """Process a single dataset with all models"""
    results = []
    
    # Prepare data
    X_train, y_train = prepare_data(train_df)
    X_test, y_test = prepare_data(test_df)
    
    # print(f"Data shapes - Train: {train_df.shape}, Test: {test_df.shape}")
    
    # Train and evaluate all models
    for model_name, model in models.items():
        result = train_and_evaluate(
            model_name, model, 
            X_train, y_train, 
            X_test, y_test,
            param_grids[model_name]
        )
        results.append({**scenario_info, **result})
    
    return results

In [11]:
datasets['baselines']['full']['region_holdout']['europe_n_central_asia']['train'].shape

(275, 82)

In [12]:
results = []
n_runs = 5 # number of runs for statistical comparison

In [13]:
import warnings

warnings.filterwarnings("ignore", message="Ill-conditioned matrix", category=UserWarning)

try:
    from sklearn.utils._testing import LinAlgWarning
except ImportError:
    from scipy.linalg import LinAlgWarning
except ImportError:
     LinAlgWarning = UserWarning # Fallback if LinAlgWarning is not easily accessible

warnings.filterwarnings("ignore", category=LinAlgWarning)

In [14]:
%%time
for run in range(n_runs):
    seed = random.randint(0, 100)  # Random seed for each run
    models = init_models(seed)  # Initialize models with the new seed
    print(f"Run {run + 1}/{n_runs} with seed {seed}")
    for scenario_type, scenario_dict in datasets.items():        # e.g., 'baselines', 'diversified'
        for scenario_name, splits in scenario_dict.items():      # e.g., 'full', 'constrained', 'max_balanced', etc.
            for split_type, data_group in splits.items():        # 'within_sample', 'region_holdout', 'size_holdout'
                # if split_type == 'original_data':
                #     continue  # skip original data entries

                # Handle different split types
                if split_type == 'within_sample':
                    data = data_group['within_sample'] # for within_sample, the data structure is one level deeper

                    scenario_info = {
                        'run': run,
                        'scenario_type': scenario_type,
                        'scenario': scenario_name,
                        'split': split_type,
                        'context': 'within_sample',
                    }
                    
                    results.extend(process_dataset(
                        data['train'], 
                        data['test'], 
                        models,
                        param_grids,
                        scenario_info
                    ))
                
                elif split_type in ['region_holdout', 'size_holdout']:
                    for context_name, data in data_group.items():
                        if 'train' not in data or 'test' not in data:
                            continue # skip if train/test not available

                        scenario_info = {
                            'run': run,
                            'scenario_type': scenario_type,
                            'scenario': scenario_name,
                            'split': split_type,
                            'context': context_name
                        }
                        
                        results.extend(process_dataset(
                            data['train'],
                            data['test'],
                            models,
                            param_grids,
                            scenario_info
                        ))

results_df = pd.DataFrame(results)

Run 1/5 with seed 49


l:\Programs\anaconda\envs\esg_pred\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


KeyboardInterrupt: 

In [17]:
results_df = pd.DataFrame(results)

In [18]:
results_df.head(25)

,run,scenario_type,scenario,split,context,model,RMSE,MAE,R2,Stability
0,0,baselines,full,within_sample,within_sample,Ridge,3.605126e+01,1.317664e+01,-2.053286e+01,-0.000886
1,0,baselines,full,within_sample,within_sample,RandomForest,6.853874e+00,5.183024e+00,2.217250e-01,0.042080
2,0,baselines,full,within_sample,within_sample,XGBoost,6.959850e+00,5.228697e+00,1.974713e-01,0.032714
3,0,baselines,full,within_sample,within_sample,MLP,1.995883e+08,3.212502e+07,-6.599803e+14,0.000613
4,0,baselines,full,region_holdout,east_asia_n_pacific,Ridge,1.221359e+02,6.728334e+01,-2.619003e+02,0.000968
5,0,baselines,full,region_holdout,east_asia_n_pacific,RandomForest,8.878998e+00,7.331653e+00,-3.894165e-01,0.045592
6,0,baselines,full,region_holdout,east_asia_n_pacific,XGBoost,8.221527e+00,6.583589e+00,-1.912682e-01,0.048606
7,0,baselines,full,region_holdout,east_asia_n_pacific,MLP,1.544044e+10,1.993592e+09,-4.201690e+18,0.000014
8,0,baselines,full,region_holdout,europe_n_central_asia,Ridge,7.365533e+00,5.843953e+00,5.464359e-02,-0.000264
9,0,baselines,full,region_holdout,europe_n_central_asia,RandomForest,6.038049e+00,4.663827e+00,3.646976e-01,0.073040


In [22]:
results_df.shape

(700, 10)

In [19]:
results_df['R2'].max()

0.9012051489960696

In [11]:
results_df = pd.read_csv('..\data\experimental\experiment_results_3.csv')

In [12]:
results_df

,run,scenario_type,scenario,split,context,model,RMSE,MAE,R2,Stability
0,0,baselines,full,within_sample,within_sample,Ridge,3.605126e+01,1.317664e+01,-2.053286e+01,-0.000886
1,0,baselines,full,within_sample,within_sample,RandomForest,6.853874e+00,5.183024e+00,2.217250e-01,0.042080
2,0,baselines,full,within_sample,within_sample,XGBoost,6.959850e+00,5.228697e+00,1.974713e-01,0.032714
3,0,baselines,full,within_sample,within_sample,MLP,1.995883e+08,3.212502e+07,-6.599803e+14,0.000613
4,0,baselines,full,region_holdout,east_asia_n_pacific,Ridge,1.221359e+02,6.728334e+01,-2.619003e+02,0.000968
...,...,...,...,...,...,...,...,...,...,...
695,4,diversified,median_balanced,size_holdout,Mid-Cap,MLP,1.501225e+09,2.323859e+08,-4.152267e+16,0.000145
696,4,diversified,median_balanced,size_holdout,Small-Cap,Ridge,1.530604e+01,9.344703e+00,-1.927470e+00,-0.000404
697,4,diversified,median_balanced,size_holdout,Small-Cap,RandomForest,8.895846e+00,6.472343e+00,1.112487e-02,-0.011056
698,4,diversified,median_balanced,size_holdout,Small-Cap,XGBoost,9.466702e+00,6.653255e+00,-1.198616e-01,-0.015979


# Statistical Comparisons

In [37]:
results_df[results_df.scenario_type=='diversified'].scenario.unique()

array(['max_balanced', 'median_balanced'], dtype=object)

In [42]:
mask_base = (
    (results_df.scenario_type=='baselines') &
    (results_df.scenario=='full') &
    (results_df.split=='within_sample') &
    (results_df.context=='within_sample') &
    (results_df.model=='MLP')
)

mask_div = (
    (results_df.scenario_type=='diversified') &
    (results_df.scenario=='median_balanced') &
    (results_df.split=='within_sample') &
    (results_df.context=='within_sample') &
    (results_df.model=='MLP')
)

In [43]:
results_df[mask_base].sort_values('run')['RMSE']

3      1.995883e+08
143    1.995883e+08
283    1.995883e+08
423    1.995883e+08
563    1.995883e+08
Name: RMSE, dtype: float64

In [44]:
results_df[mask_div].sort_values('run')['RMSE']

103    2.368309e+08
243    2.368309e+08
383    2.368309e+08
523    2.368309e+08
663    2.368309e+08
Name: RMSE, dtype: float64

In [13]:
def paired_t_tests(results, baseline_scenario='full', diversified_scenarios=None):
    """
    Perform paired t-tests comparing baseline 'baselines' vs each diversified scenario.
    Returns a DataFrame of test statistics and p-values for each combination of:
    split, context, model, diversified_scenario.
    """
    if diversified_scenarios is None:
        diversified_scenarios = results[results.scenario_type=='diversified']['scenario'].unique()
    
    rows = []
    for split in results['split'].unique():
        for context in results[results['split']==split]['context'].unique():
            for model in results['model'].unique():
                # baseline RMSEs
                mask_base = (
                    (results.scenario_type=='baselines') &
                    (results.scenario=='full') &
                    (results.split==split) &
                    (results.context==context) &
                    (results.model==model)
                )
                base_vals = results[mask_base].sort_values('run')['RMSE'].values
                for div_scenario in diversified_scenarios:
                    mask_div = (
                        (results.scenario_type=='diversified') &
                        (results.scenario==div_scenario) &
                        (results.split==split) &
                        (results.context==context) &
                        (results.model==model)
                    )
                    div_vals = results[mask_div].sort_values('run')['RMSE'].values
                    # require equal-length arrays
                    if len(base_vals)==len(div_vals) and len(base_vals)>1:
                        stat, p = ttest_rel(base_vals, div_vals)
                        rows.append({
                            'split': split,
                            'context': context,
                            'model': model,
                            'diversified_scenario': div_scenario,
                            't_stat': stat,
                            'p_value': p
                        })
    return pd.DataFrame(rows)

paired_results_df = paired_t_tests(results_df)

l:\Programs\anaconda\envs\esg_pred\Lib\site-packages\scipy\stats\_axis_nan_policy.py:430: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  return hypotest_fun_in(*args, **kwds)


In [14]:
paired_results_df

,split,context,model,diversified_scenario,t_stat,p_value
0,within_sample,within_sample,Ridge,max_balanced,inf,0.000000e+00
1,within_sample,within_sample,Ridge,median_balanced,1.603775e+16,9.069370e-65
2,within_sample,within_sample,RandomForest,max_balanced,inf,0.000000e+00
3,within_sample,within_sample,RandomForest,median_balanced,1.697340e+16,7.228968e-65
4,within_sample,within_sample,XGBoost,max_balanced,inf,0.000000e+00
...,...,...,...,...,...,...
75,size_holdout,Small-Cap,RandomForest,median_balanced,-inf,0.000000e+00
76,size_holdout,Small-Cap,XGBoost,max_balanced,-inf,0.000000e+00
77,size_holdout,Small-Cap,XGBoost,median_balanced,-inf,0.000000e+00
78,size_holdout,Small-Cap,MLP,max_balanced,-inf,0.000000e+00


In [46]:
# ----------------------------------------
# 2. Friedman + Tukey HSD: Multiple-model comparison
#    on diversified within-sample 'full' (i.e., context=='within_sample')
# ----------------------------------------

def multi_model_comparison(results, scenario_type='diversified', scenario_name='max_balanced'):
    """
    Perform Friedman test across models on within-sample for a given diversified scenario,
    followed by Tukey HSD for post-hoc pairwise comparisons.
    Returns a tuple: (friedman_stat, friedman_p, tukey_df)
    """
    # Filter for within-sample context
    mask = (
        (results.scenario_type==scenario_type) &
        (results.scenario==scenario_name) &
        (results.split=='within_sample')
    )
    df_sub = results[mask]
    
    # Pivot: rows=runs, columns=models
    pivot = df_sub.pivot(index='run', columns='model', values='RMSE')
    models = pivot.columns.tolist()
    
    # Friedman test
    data = [pivot[m].values for m in models]
    stat, p = friedmanchisquare(*data)
    
    # Tukey HSD
    tukey = pairwise_tukeyhsd(endog=df_sub['RMSE'], groups=df_sub['model'], alpha=0.05)
    tukey_df = pd.DataFrame(data=tukey._results_table.data[1:], 
                            columns=tukey._results_table.data[0])
    
    return stat, p, tukey_df

# Apply multi-model comparison for each diversified scenario_name
multi_results = []
for scenario_name in results_df[results_df.scenario_type=='diversified']['scenario'].unique():
    stat, p, tukey_df = multi_model_comparison(results_df, scenario_type='diversified', scenario_name=scenario_name)
    # store summary
    summary = {
        'scenario': scenario_name,
        'friedman_stat': stat,
        'friedman_p': p
    }
    multi_results.append(summary)
    # display Tukey results per scenario
    # tools.display_dataframe_to_user(f"Tukey HSD Post-hoc ({scenario_name})", tukey_df)

multi_results_df = pd.DataFrame(multi_results)

l:\Programs\anaconda\envs\esg_pred\Lib\site-packages\statsmodels\sandbox\stats\multicomp.py:1300: RuntimeWarning: divide by zero encountered in divide
  st_range = np.abs(meandiffs) / std_pairs #studentized range statistic


In [47]:
multi_results_df

,scenario,friedman_stat,friedman_p
0,max_balanced,15.0,0.001817
1,median_balanced,15.0,0.001817


In [48]:
tukey_df

,group1,group2,meandiff,p-adj,lower,upper,reject
0,MLP,RandomForest,-2.368309e+08,0.0,-2.368309e+08,-2.368309e+08,True
1,MLP,Ridge,-2.368309e+08,0.0,-2.368309e+08,-2.368309e+08,True
2,MLP,XGBoost,-2.368309e+08,0.0,-2.368309e+08,-2.368309e+08,True
3,RandomForest,Ridge,2.593000e+00,0.0,2.593000e+00,2.593000e+00,True
4,RandomForest,XGBoost,2.481000e-01,0.0,2.481000e-01,2.481000e-01,True
5,Ridge,XGBoost,-2.345000e+00,0.0,-2.345000e+00,-2.345000e+00,True


In [54]:
# Extract within-sample RMSE per scenario/model
within = results_df[results_df['split'] == 'within_sample'][['scenario_type', 'scenario', 'model', 'RMSE']]
within = within.rename(columns={'RMSE': 'RMSE_within'})

# Merge to get RMSE_within alongside all rows
merged = results_df.merge(within, on=['scenario_type', 'scenario', 'model'], how='left')

# Compute Cross-Context Generalization Score:
#    Transfer Ratio = (RMSE_holdout / RMSE_within)
#    For within-sample rows, set NaN
merged['TransferRatio'] = np.where(
    merged['split'] == 'within_sample',
    np.nan,
    merged['RMSE_within'] / merged['RMSE']
)

In [55]:
merged

,run,scenario_type,scenario,split,context,model,RMSE,MAE,R2,Stability,RMSE_within,TransferRatio
0,0,baselines,full,within_sample,within_sample,Ridge,3.605126e+01,1.317664e+01,-2.053286e+01,-0.000886,3.605126e+01,NaN
1,0,baselines,full,within_sample,within_sample,Ridge,3.605126e+01,1.317664e+01,-2.053286e+01,-0.000886,3.605126e+01,NaN
2,0,baselines,full,within_sample,within_sample,Ridge,3.605126e+01,1.317664e+01,-2.053286e+01,-0.000886,3.605126e+01,NaN
3,0,baselines,full,within_sample,within_sample,Ridge,3.605126e+01,1.317664e+01,-2.053286e+01,-0.000886,3.605126e+01,NaN
4,0,baselines,full,within_sample,within_sample,Ridge,3.605126e+01,1.317664e+01,-2.053286e+01,-0.000886,3.605126e+01,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
3495,4,diversified,median_balanced,size_holdout,Small-Cap,MLP,1.249244e+08,3.394963e+07,-1.950120e+14,-0.000059,2.368309e+08,1.895794
3496,4,diversified,median_balanced,size_holdout,Small-Cap,MLP,1.249244e+08,3.394963e+07,-1.950120e+14,-0.000059,2.368309e+08,1.895794
3497,4,diversified,median_balanced,size_holdout,Small-Cap,MLP,1.249244e+08,3.394963e+07,-1.950120e+14,-0.000059,2.368309e+08,1.895794
3498,4,diversified,median_balanced,size_holdout,Small-Cap,MLP,1.249244e+08,3.394963e+07,-1.950120e+14,-0.000059,2.368309e+08,1.895794


In [62]:
merged.groupby('model').agg(metric=('R2','std'))

,metric
model,
MLP,6.574459e+18
RandomForest,2.770499e-01
Ridge,9.518219e+02
XGBoost,2.394101e-01


In [64]:
merged.groupby(['scenario_type', 'scenario', 'split', 'context', 'model']).agg(
        RMSE_mean=('RMSE', 'mean'),
        RMSE_std=('RMSE', 'std'),
        MAE_mean=('MAE', 'mean'),
        MAE_std=('MAE', 'std'),
        R2_mean=('R2', 'mean'),
        R2_std=('R2', 'std'),
        Stability_mean=('Stability', 'mean'),
        Stability_std=('Stability', 'std'),
        TransferRatio_mean=('TransferRatio', 'mean'),
        TransferRatio_std=('TransferRatio', 'std')
    )

RMSE_mean  \
scenario_type scenario        split          context               model                        
baselines     constrained     region_holdout europe_n_central_asia MLP           2.006528e+07   
                                                                   RandomForest  6.007674e+00   
                                                                   Ridge         1.912827e+01   
                                                                   XGBoost       6.614969e+00   
                                             north_america         MLP           1.547741e+08   
...                                                                                       ...   
diversified   median_balanced size_holdout   Small-Cap             XGBoost       9.466702e+00   
                              within_sample  within_sample         MLP           2.368309e+08   
                                                                   RandomForest  4.969448e+00   
                                                                   Ridge         7.562487e+00   
                                                                   XGBoost       5.217513e+00   

                                                                                 RMSE_std  \
scenario_type scenario        split          context               model                    
baselines     constrained     region_holdout europe_n_central_asia MLP                0.0   
                                                                   RandomForest       0.0   
                                                                   Ridge              0.0   
                                                                   XGBoost            0.0   
                                             north_america         MLP                0.0   
...                                                                                   ...   
diversified   median_balanced size_holdout   Small-Cap             XGBoost            0.0   
                              within_sample  within_sample         MLP                0.0   
                                                                   RandomForest       0.0   
                                                                   Ridge              0.0   
                                                                   XGBoost            0.0   

                                                                                     MAE_mean  \
scenario_type scenario        split          context               model                        
baselines     constrained     region_holdout europe_n_central_asia MLP           5.360231e+06   
                                                                   RandomForest  4.628622e+00   
                                                                   Ridge         1.104391e+01   
                                                                   XGBoost       5.312802e+00   
                                             north_america         MLP           2.744272e+07   
...                                                                                       ...   
diversified   median_balanced size_holdout   Small-Cap             XGBoost       6.653255e+00   
                              within_sample  within_sample         MLP           6.143134e+07   
                                                                   RandomForest  3.191702e+00   
                                                                   Ridge         5.221922e+00   
                                                                   XGBoost       3.260219e+00   

                                                                                 MAE_std  \
scenario_type scenario        split          context               model                   
baselines     constrained     region_holdout europe_n_central_asia MLP               0.0   
                                                                   RandomF

In [56]:
agg_df = (
    merged
    .groupby(['scenario_type', 'scenario', 'split', 'context', 'model'])
    .agg(
        RMSE_mean=('RMSE', 'mean'),
        RMSE_std=('RMSE', 'std'),
        MAE_mean=('MAE', 'mean'),
        MAE_std=('MAE', 'std'),
        R2_mean=('R2', 'mean'),
        R2_std=('R2', 'std'),
        Stability_mean=('Stability', 'mean'),
        Stability_std=('Stability', 'std'),
        TransferRatio_mean=('TransferRatio', 'mean'),
        TransferRatio_std=('TransferRatio', 'std')
    )
    .reset_index()
)

In [57]:
agg_df

,scenario_type,scenario,split,context,model,RMSE_mean,RMSE_std,MAE_mean,MAE_std,R2_mean,R2_std,Stability_mean,Stability_std,TransferRatio_mean,TransferRatio_std
0,baselines,constrained,region_holdout,europe_n_central_asia,MLP,2.006528e+07,0.0,5.360231e+06,0.0,-6.975671e+12,0.0,0.372022,0.012095,6.138905,0.0
1,baselines,constrained,region_holdout,europe_n_central_asia,RandomForest,6.007674e+00,0.0,4.628622e+00,0.0,3.746715e-01,0.0,0.090024,0.007817,0.912024,0.0
2,baselines,constrained,region_holdout,europe_n_central_asia,Ridge,1.912827e+01,0.0,1.104391e+01,0.0,-5.339381e+00,0.0,0.003921,0.002954,0.429888,0.0
3,baselines,constrained,region_holdout,europe_n_central_asia,XGBoost,6.614969e+00,0.0,5.312802e+00,0.0,2.418569e-01,0.0,0.040649,0.006152,0.898587,0.0
4,baselines,constrained,region_holdout,north_america,MLP,1.547741e+08,0.0,2.744272e+07,0.0,-5.103642e+14,0.0,0.001567,0.000391,0.795862,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,diversified,median_balanced,size_holdout,Small-Cap,XGBoost,9.466702e+00,0.0,6.653255e+00,0.0,-1.198616e-01,0.0,-0.012620,0.004374,0.551144,0.0
136,diversified,median_balanced,within_sample,within_sample,MLP,2.368309e+08,0.0,6.143134e+07,0.0,-1.025021e+15,0.0,0.000369,0.000268,NaN,NaN
137,diversified,median_balanced,within_sample,within_sample,RandomForest,4.969448e+00,0.0,3.191702e+00,0.0,5.486927e-01,0.0,0.330500,0.007108,NaN,NaN
138,diversified,median_balanced,within_sample,within_sample,Ridge,7.562487e+00,0.0,5.221922e+00,0.0,-4.516628e-02,0.0,0.015677,0.005752,NaN,NaN
